In [ ]:
import os
import sys
import re
import time
import matplotlib
import matplotlib.pyplot as plt
import random

parentPath = os.path.abspath("../..")
if parentPath not in sys.path:
    sys.path.insert(0, parentPath)

import model as modellib
import visualize
from model import log

%matplotlib inline 

# Root directory of the project
ROOT_DIR = parentPath

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
# MODEL_DIR = os.path.join("/data/orestisz/logs")

SCENENN_DIR = "/external_datasets/SceneNet_RGBD"
# SCENENET_MODEL_PATH = os.path.join(MODEL_DIR, "scenenet20180428T1942/mask_rcnn_scenenet_0576.h5")
# SCENENET_MODEL_PATH = os.path.join(MODEL_DIR, "scenenet_coco_rgb20180428T1942/mask_rcnn_scenenet_coco_rgb_0600.h5")
# SCENENET_MODEL_PATH = os.path.join(MODEL_DIR, "scenenet_classes20180428T1942/mask_rcnn_scenenet_classes_0700.h5")
SCENENET_MODEL_PATH = os.path.join(MODEL_DIR, "scenenet_coco_rgb_classes20180428T1942/mask_rcnn_scenenet_coco_rgb_classes_0700.h5")
# MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# import SceneNet
from dataset import *

def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# Validation dataset
dataset_test = Dataset()
dataset_test.load("validation")
dataset_test.prepare()

## Detection

In [ ]:
class InferenceConfig(Config):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()

In [ ]:
# Test on a random image
image_id = random.choice(dataset_test.image_ids)
# image_id = 11173
# image_id = 72081
# image_id = 199013
print("image_id", image_id)

original_image, image_meta, class_ids, bbox, mask=\
    modellib.load_image_gt(dataset_test, config, 
                           image_id, use_mini_mask=False)
print(dataset_test.image_info[image_id]['path'])

image = original_image[:, :, 0:3]
log("image", image)
log("image_meta", image_meta)
log("bbox", bbox)
log("mask", mask)

if len(config.MODE) > 3:
    depth = original_image[:, :, 3]
    depth = depth.astype(np.uint8)
    depth_norm = depth / np.max(depth)
    log("depth", depth)
    rgbd = np.dstack((
        image[:, :, 0] * depth_norm,
        image[:, :, 1] * depth_norm,
        image[:, :, 2] * depth_norm))

    ax = get_ax()
    ax.axis('off')
    ax.imshow(depth_norm)

    ax = get_ax()
    ax.axis('off')
    ax.imshow(rgbd.astype(np.uint8))
else:
    ax = get_ax()
    ax.axis('off')
    ax.imshow(image.astype(np.uint8))

visualize.display_top_masks(image, mask, class_ids, dataset_test.class_names, limit=1)
visualize.display_instances(image, bbox, mask, class_ids, dataset_test.class_names, figsize=(8, 8))

In [ ]:
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", config=config, model_dir=MODEL_DIR)
model.load_weights(SCENENET_MODEL_PATH, by_name=True)

In [ ]:
# %%script false
if False: # test transfer weights rgb -> rgbd
    # get weights from first convolution
    conv1 = model.keras_model.get_layer("conv1")
    kernel_rgb, bias = conv1.get_weights()

    # reload model in rgb-d mode
    config_rgbd = InferenceConfig()
    config_rgbd.MODE = "RGBD"
    config_rgbd.MEAN_PIXEL = np.array([123.7, 116.8, 103.9, 255 / 2])
    config_rgbd.IMAGE_SHAPE = np.array([config.IMAGE_MAX_DIM, config.IMAGE_MAX_DIM, 4])
    model_rgbd = modellib.MaskRCNN(mode="inference", config=config_rgbd, model_dir=MODEL_DIR)
    model_rgbd.load_weights(SCENENET_MODEL_PATH, by_name=True, exclude=["conv1"])

    # set weights on first convolution from rgb model
    conv1 = model_rgbd.keras_model.get_layer("conv1")
    kernel_rgbd = np.concatenate((kernel_rgb, np.mean(kernel_rgb, keepdims=True, axis=2)), axis=2)
#     kernel_rgbd = np.concatenate((kernel_rgb, np.zeros((7, 7, 1, 64))), axis=2)
    conv1.set_weights([kernel_rgbd, bias])
    original_image, _, _, _, _ = modellib.load_image_gt(dataset_test, config_rgbd, image_id, use_mini_mask=False)
    
    start = time.clock()
    r = model_rgbd.detect([original_image])[0]
    print(time.clock() - start)
    visualize.display_instances(original_image[:, :, 0:3], r['rois'], r['masks'], r['class_ids'], 
                        dataset_test.class_names, r['scores'], ax=get_ax())
else:
    delta = []
    for idx, image_id in enumerate(dataset_test.image_ids[0::1000]):
        original_image, image_meta, class_ids, bbox, mask=\
        modellib.load_image_gt(dataset_test, config, 
                           image_id, use_mini_mask=False)

        image = original_image[:, :, 0:3]

        start = time.clock()
        r = model.detect([original_image])[0]
        print(time.clock() - start)
        if (idx):
            delta.append(time.clock() - start)
        visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_test.class_names, r['scores'], ax=get_ax(),
                            title="sceneNet_classes/{}_{}".format(image_id, config.MODE))
        
    print("avarage delta: ", np.mean(delta))

## Evaluation

In [ ]:
%%script false
import utils
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = dataset_test.image_ids[0::30]
APs_50 = []
APs_75 = []
APs_50_95 = []
for i, image_id in enumerate(image_ids):
    # Load image and ground truth data
    image_path = dataset_test.image_info[image_id]['path']
    try:
        image, image_meta, gt_class_id, gt_bbox, gt_mask  =\
            modellib.load_image_gt(dataset_test, config,
                                   image_id, use_mini_mask=False)
        # Run object detection
        results = model.detect([image], verbose=0)
        r = results[0]
        # Compute AP
        AP_50, precisions, recalls, overlaps =\
            utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                             r["rois"], r["class_ids"], r["scores"], r['masks'], iou_threshold=0.5)
        AP_75, precisions, recalls, overlaps =\
            utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                             r["rois"], r["class_ids"], r["scores"], r['masks'], iou_threshold=0.75)
        AP_50_95 =\
            utils.compute_ap_range(gt_bbox, gt_class_id, gt_mask,
                             r["rois"], r["class_ids"], r["scores"], r['masks'], verbose=0)
        
        APs_50.append(AP_50)
        APs_75.append(AP_75)
        APs_50_95.append(AP_50_95)

#         print('{}: {}: {}, {}, {}'.format(i + 1, image_path, AP_50, AP_75, AP_50_95))
    except:
        print("Error processing image {}".format(image_path))

print("mAP@IoU=0.5: ", np.mean(APs_50))
print("mAP@IoU=0.75: ", np.mean(APs_75))
print("mAP@IoU=0.5:0.95: ", np.mean(APs_50_95))
